In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import Imputer
from xgboost import XGBRegressor

In [2]:
iowa_file_path = '../input/train.csv'
home_data = pd.read_csv(iowa_file_path)

In [3]:
data = home_data.drop(['MiscFeature', 'Fence', 'PoolQC', 'Alley', 'FireplaceQu'], axis=1)

features = data.columns

cols = data[features]
numeric_cols = cols.select_dtypes(include=['Int64','float64'])
numeric_cols.fillna(numeric_cols.mean(), inplace=True)

cat_cols = cols.select_dtypes(include=['object'])
print(cat_cols.head(5))

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


  MSZoning Street LotShape      ...      PavedDrive SaleType SaleCondition
0       RL   Pave      Reg      ...               Y       WD        Normal
1       RL   Pave      Reg      ...               Y       WD        Normal
2       RL   Pave      IR1      ...               Y       WD        Normal
3       RL   Pave      IR1      ...               Y       WD       Abnorml
4       RL   Pave      IR1      ...               Y       WD        Normal

[5 rows x 38 columns]


In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_cols = cat_cols.astype(str)
cat_cols = cat_cols.apply(le.fit_transform)

X = pd.concat([numeric_cols, cat_cols], axis = 1)
X = X.drop(['SalePrice'], axis = 1)
y = data['SalePrice']

print(X.head())

   Id  MSSubClass      ...        SaleType  SaleCondition
0   1          60      ...               8              4
1   2          20      ...               8              4
2   3          60      ...               8              4
3   4          70      ...               8              0
4   5          60      ...               8              4

[5 rows x 75 columns]


# XG Boost Model and Validation

In [5]:
from sklearn.metrics import mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)

# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 
          'nthread':[4],
          'gamma':[i/10.0 for i in range(3,6)],  
          'learning_rate': [.03, 0.05, .07, 0.1],
          'subsample':[i/10.0 for i in range(6,11)],
          'min_child_weight': [4],
          'silent': [1],
          'colsample_bytree':[i/10.0 for i in range(6,11)], 
          'max_depth': [2,3,4],
          'n_estimators': [1000]}

my_XGB_model = XGBRegressor(nthread=-1,objective='reg:linear',missing=None,seed=8)
grid = RandomizedSearchCV(my_XGB_model, params, n_jobs=1,n_iter=10)
grid.fit(X_train, y_train, verbose=False)
print(grid.best_params_)
XGB_predictions = grid.predict(X_test)

print("MAE for best value: ", mean_absolute_error(XGB_predictions, y_test))

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getat

{'subsample': 0.6, 'silent': 1, 'nthread': 4, 'n_estimators': 1000, 'min_child_weight': 4, 'max_depth': 2, 'learning_rate': 0.03, 'gamma': 0.5, 'colsample_bytree': 0.7}
MAE for best value:  16050.598458904109


In [6]:
test_file_path = '../input/test.csv'
test_data = pd.read_csv(test_file_path)
test_data = test_data.drop(['MiscFeature', 'Fence', 'PoolQC', 'Alley', 'FireplaceQu'], axis=1)

num_test_data = test_data.select_dtypes(include=['Int64','float64'])
cat_test_data = test_data.select_dtypes(include=['object'])
cat_test_data = cat_test_data.astype(str)
cat_test_data = cat_test_data.apply(LabelEncoder().fit_transform)
test_data = pd.concat([num_test_data, cat_test_data], axis=1)
print(test_data.head())
final_predictions = grid.best_estimator_.predict(test_data)

     Id  MSSubClass      ...        SaleType  SaleCondition
0  1461          20      ...               8              4
1  1462          20      ...               8              4
2  1463          60      ...               8              4
3  1464          60      ...               8              4
4  1465         120      ...               8              4

[5 rows x 75 columns]


#  Write output CSV file of predictions from test data for contest submission

In [7]:
output = pd.DataFrame({'Id': test_data.Id, 'SalePrice': final_predictions})
output.to_csv('submission.csv', index=False)